In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import utils
import time

In [3]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

cuda


In [4]:
from utils import check_cifar_dataset_exists
data_path=check_cifar_dataset_exists()

train_data=torch.load(data_path+'cifar/train_data.pt')
train_label=torch.load(data_path+'cifar/train_label.pt')
test_data=torch.load(data_path+'cifar/test_data.pt')
test_label=torch.load(data_path+'cifar/test_label.pt')

print(train_data.size())
print(test_data.size())

CIFAR dataset missing - downloading...
Files already downloaded and verified
torch.Size([50000, 3, 32, 32])
torch.Size([10000, 3, 32, 32])


In [5]:
mean= train_data.mean()

print(mean)

tensor(0.4734)


In [6]:
std= train_data.std()

print(std)

tensor(0.2516)


In [ ]:
class Stem(nn.Module):

    def __init__(self):

        super(Stem, self).__init__()
        
        #----------------------- stem block start ----------------------------
        
        #3 x 299 x 299 --> 32 x 149 x 149  , VALID Padding 
        self.conv1a = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=0 ) 
        
        #32 x 149 x 149 --> 32 x 147 x 147  , VALID Padding 
        self.conv1b = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0 )
        
        #32 x 147 x 147 --> 64 x 147 x 147  , SAME Padding 
        self.conv1c = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1 )
        
        #======================== Filter concat 1 =============================
        
        #kernel size = 3, VALID Padding
        self.pool1  = nn.MaxPool2d(3, stride=2, padding=0 )
        
        #64 x 147 x 147 --> 96 x --- x ---  , VALID Padding   Find out the size of output
        self.conv2a = nn.Conv2d(64, 96, kernel_size=3, stride=2, padding=0 )
        
        #======================== Filter concat 1 =============================
        
        #======================== Filter concat 2 =============================       
        
        #160 x 73 x 73 --> 64 x --- x ---  , SAME Padding   Find out the size of output
        self.conv3a = nn.Conv2d(160, 64, kernel_size=1, padding=1 )
        
        #64 x --- x --- --> 96 x --- x ---  , VALID Padding   Find out the size of output
        self.conv3b = nn.Conv2d(64, 96, kernel_size=3, padding=0 )
        
        #160 x 73 x 73 --> 64 x --- x ---  , SAME Padding   Might be combined with self.conv1e1
        self.conv4a = nn.Conv2d(160, 64, kernel_size=1, padding=1 )
        
        #64 x --- x --- --> 64 x --- x ---  , SAME Padding   Find out the size of output
        self.conv4b = nn.Conv2d(64, 64, kernel_size=[7,1], padding=1 )
        
        #64 x --- x --- --> 64 x --- x ---  , SAME Padding   Find out the size of output
        self.conv4c = nn.Conv2d(64, 64, kernel_size=[1,7], padding=1 )
        
        #64 x --- x --- --> 96 x --- x ---  , VALID Padding   Might be combined with self.conv1e2
        self.conv4d = nn.Conv2d(64, 96, kernel_size=3, padding=0 )
        
        #======================== Filter concat 2 ============================= 
        
        #======================== Filter concat 3 =============================
        
        #192 x 71 x 71 --> 384 x 35 x 35  , VALID Padding 
        self.conv5a = nn.Conv2d(192, 384, kernel_size=3, padding=0 )
         
        #kernel size = 1, VALID Padding
        self.pool2  = nn.MaxPool2d(1, stride=2, padding=0 )
        
        #======================== Filter concat 3 =============================
        
        #----------------------- stem block finish ----------------------------
        
    def forward(self, x):

        # block 1:         3 x 32 x 32 --> 64 x 16 x 16
        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.conv1c(x)
        x = F.relu(x)
        
        xP = self.pool1(x)
        xC = self.conv2a(x)
        xC = F.relu(xC)
        xFC1 = torch.cat((xP, xC), 1)
        
        y = self.conv3a(xFC1)
        y = F.relu(y)
        y = self.conv3b(y)
        y = F.relu(y)

        z = self.conv4a(xFC1)
        z = F.relu(z)
        z = self.conv4b(z)
        z = F.relu(z)
        z = self.conv4c(xFC1)
        z = F.relu(z)
        z = self.conv4d(z)
        z = F.relu(z)
        
        # Above code or this one?
        #z = self.conv3a(xFC1)
        #z = F.relu(z)
        #z = self.conv4b(z)
        #z = F.relu(z)
        #z = self.conv4c(xFC1)
        #z = F.relu(z)
        #z = self.conv3b(z)
        #z = F.relu(z)
        
        xFC2 = torch.cat((y, z), 1)
        
        xP = self.pool2(xFC2)
        xC = self.conv5a(xFC2)
        xC = F.relu(xC)
        xFC3 = torch.cat((xP, xC), 1)   
        
        return xFC3

In [ ]:
class InceptionA(nn.Module):

    def __init__(self):

        super(InceptionA, self).__init__()
        
        #----------------------- InceptionA block start ----------------------------
        
        #======================== Filter concat =============================
        
        #block 1
        #kernel size = 1, SAME Padding, OUTPUT = 384 Verify this
        self.pool1 = nn.AvgPool2d(1, padding=1 )
        
        #384 x --- x --- --> 96 x --- x ---  , SAME Padding 
        self.conv1a = nn.Conv2d(384, 96, kernel_size=1, padding=1 ) 
        
        #block 2
        #384 x 35 x 35 --> 96 x --- x ---  , SAME Padding 
        self.conv2a = nn.Conv2d(384, 96, kernel_size=1, padding=1 ) 
        
        #block 3
        #384 x 35 x 35 --> 64 x --- x ---  , SAME Padding 
        self.conv3a = nn.Conv2d(384, 64, kernel_size=1, padding=1 )
        
        #64 x --- x --- --> 96 x --- x ---  , SAME Padding 
        self.conv3b = nn.Conv2d(64, 96, kernel_size=3, padding=1 )
        
        #block 4
        #384 x 35 x 35 --> 64 x --- x ---  , SAME Padding 
        self.conv4a = nn.Conv2d(384, 64, kernel_size=1, padding=1 )

        #64 x --- x --- --> 96 x --- x ---  , SAME Padding 
        self.conv4b = nn.Conv2d(64, 96, kernel_size=3, padding=1 )
        
        #96 x --- x --- --> 96 x --- x ---  , SAME Padding 
        self.conv4c = nn.Conv2d(96, 96, kernel_size=3, padding=1 )
        
        
        #======================== Filter concat =============================
        
        #----------------------- InceptionA block finish ----------------------------
        
    def forward(self, x):

        # block 1:
        y = self.pool1(x)
        y = self.conv1a(y)
        y = F.relu(y)   # Do we need Relu here (after last operation)?
        
        # block 2:
        z = self.conv2a(x)
        z = F.relu(z) 
        
        #block 3:
        w = self.conv3a(x)
        w = F.relu(w)
        w = self.conv3b(w)
        w = F.relu(w)
        
        #block 4:
        v = self.conv4a(x)
        v = F.relu(v)
        v = self.conv4b(v)
        v = F.relu(v)
        
        xFC = torch.cat((y, z, w, v), 1)
        
        return xFC

In [ ]:
class InceptionB(nn.Module):

    def __init__(self):

        super(InceptionB, self).__init__()
        
        #----------------------- InceptionB block start ----------------------------
        
        #======================== Filter concat =============================
        
        #block 1
        #kernel size = 1, SAME Padding, OUTPUT = 384 Verify this
        self.pool1 = nn.AvgPool2d(1, padding=1 )
        
        #1024 x --- x --- --> 128 x --- x ---  , SAME Padding 
        self.conv1a = nn.Conv2d(1024, 128, kernel_size=1, padding=1 ) 
        
        #block 2
        #1024 x 17 x 17 --> 384 x --- x ---  , SAME Padding 
        self.conv2a = nn.Conv2d(1024, 384, kernel_size=1, padding=1 ) 
        
        #block 3
        #1024 x 17 x 17 --> 192 x --- x ---  , SAME Padding 
        self.conv3a = nn.Conv2d(1024, 192, kernel_size=1, padding=1 )
        
        #192 x --- x --- --> 224 x --- x ---  , SAME Padding 
        self.conv3b = nn.Conv2d(192, 224, kernel_size=[7,1], padding=1 )
        
        #224 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv3c = nn.Conv2d(224, 256, kernel_size=[1,7], padding=1 )
        
        #block 4
        #1024 x 17 x 17 --> 192 x --- x ---  , SAME Padding 
        self.conv4a = nn.Conv2d(1024, 192, kernel_size=1, padding=1 )
        
        #192 x --- x --- --> 192 x --- x ---  , SAME Padding 
        self.conv4b = nn.Conv2d(192, 192, kernel_size=[1,7], padding=1 )
        
        #192 x --- x --- --> 224 x --- x ---  , SAME Padding 
        self.conv4c = nn.Conv2d(192, 224, kernel_size=[7,1], padding=1 )
        
        #224 x --- x --- --> 224 x --- x ---  , SAME Padding 
        self.conv4d = nn.Conv2d(224, 224, kernel_size=[1,7], padding=1 )
        
        #224 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv4e = nn.Conv2d(224, 256, kernel_size=[7,1], padding=1 )
        
        
        #======================== Filter concat =============================
        
        #----------------------- InceptionB block finish ----------------------------
        
    def forward(self, x):
        
        # block 1:
        y = self.pool1(x)
        y = self.conv1a(y)
        y = F.relu(y)
        
        # block 2:
        z = self.conv2a(x)
        z = F.relu(z) 
        
        #block 3:
        w = self.conv3a(x)
        w = F.relu(w)
        w = self.conv3b(w)
        w = F.relu(w)
        w = self.conv3c(w)
        w = F.relu(w)
        
        #block 4:
        v = self.conv4a(x)
        v = F.relu(v)
        v = self.conv4b(v)
        v = F.relu(v)
        v = self.conv4c(v)
        v = F.relu(v)
        v = self.conv4d(v)
        v = F.relu(v)
        v = self.conv4e(v)
        v = F.relu(v)
        
        xFC = torch.cat((y, z, w, v), 1)
        
        return xFC

In [ ]:
class InceptionC(nn.Module):

    def __init__(self):

        super(InceptionC, self).__init__()
        
        #----------------------- InceptionC block start ----------------------------
        
        #======================== Filter concat =============================
        
        #block 1
        #kernel size = 1, SAME Padding, OUTPUT = 384 Verify this
        self.pool1 = nn.AvgPool2d(1, padding=1 )
        
        #1536 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv1a = nn.Conv2d(1536, 256, kernel_size=1, padding=1 ) 
        
        #block 2
        #1536 x 8 x 8 --> 256 x --- x ---  , SAME Padding 
        self.conv2a = nn.Conv2d(1536, 256, kernel_size=1, padding=1 ) 
        
        #block 3
        #1536 x 8 x 8 --> 384 x --- x ---  , SAME Padding 
        self.conv3a = nn.Conv2d(1536, 384, kernel_size=1, padding=1 )
        
        #384 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv3b = nn.Conv2d(384, 256, kernel_size=[1,3], padding=1 )
        
        #384 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv3c = nn.Conv2d(384, 256, kernel_size=[3,1], padding=1 )
        
        #block 4
        #1536 x 8 x 8 --> 384 x --- x ---  , SAME Padding 
        self.conv4a = nn.Conv2d(1536, 384, kernel_size=1, padding=1 )
        
        #384 x --- x --- --> 448 x --- x ---  , SAME Padding 
        self.conv4b = nn.Conv2d(384, 448, kernel_size=[1,3], padding=1 )
        
        #448 x --- x --- --> 512 x --- x ---  , SAME Padding 
        self.conv4c = nn.Conv2d(448, 512, kernel_size=[3,1], padding=1 )
        
        #512 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv4d = nn.Conv2d(512, 256, kernel_size=[3,1], padding=1 )
        
        #512 x --- x --- --> 256 x --- x ---  , SAME Padding 
        self.conv4e = nn.Conv2d(512, 256, kernel_size=[1,3], padding=1 )
        
        
        #======================== Filter concat =============================
        
        #----------------------- InceptionC block finish ----------------------------
        
    def forward(self, x):
        
        # block 1:
        y = self.pool1(x)
        y = self.conv1a(y)
        y = F.relu(y)
        
        # block 2:
        z = self.conv2a(x)
        z = F.relu(z) 
        
        #block 3:
        w = self.conv3a(x)
        w = F.relu(w)
        w1 = self.conv3b(w)
        w1 = F.relu(w1)
        w2 = self.conv3c(w)
        w2 = F.relu(w2)
        
        #block 4:
        v = self.conv4a(x)
        v = F.relu(v)
        v = self.conv4b(v)
        v = F.relu(v)
        v = self.conv4c(v)
        v = F.relu(v)
        v1 = self.conv4d(v)
        v1 = F.relu(v1)
        v2 = self.conv4e(v)
        v2 = F.relu(v2)
        
        xFC = torch.cat((y, z, w1, w2, v1, v2), 1)
        
        return xFC

In [ ]:
class Inception_v4_convnet(nn.Module):

    def __init__(self):

        super(Inception_v4_convnet, self).__init__()
        
        

    def forward(self, x):

       
        return x